### Purpose

The intent of this notebook is to serve as a rapid testing ground for new utilities. Any logic written here should migrate to the src/ directory as proper functions.

#### Import and Constants

In [ ]:
from datetime import timedelta

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
import numpy as np
import pandas as pd
from pandas.core.generic import NDFrame  # This is the generic type that encompasses Series and DataFrame

# Importing from the local code structure
import os
import sys
module_path = os.path.abspath(os.path.join("..", "src"))
sys.path.append(module_path)
from exercise_log.constants import *
from exercise_log.dataloader import DataLoader
from exercise_log.trend import Trendsetter
from exercise_log.utils import join_with_comma

#### Load Data and Compute Trends

In [ ]:
health_metrics = DataLoader.load_health_metrics()
travel_days = DataLoader.load_travel_days()
cardio_workouts = DataLoader.load_cardio_workouts()
weight_training_workouts = DataLoader.load_weight_training_workouts()
weight_training_sets = DataLoader.load_weight_training_sets()
all_workouts = DataLoader.load_all_workouts(cardio_workouts, weight_training_workouts, travel_days)

EXTRAPOLATE_DAYS = 100
MIN_DAILY_ACTIVE_MINUTES = 22.5  # Weekly is 150, this is about 150/7
N_DAYS_TO_AVG = 8

# n-day average over a week gives a sense of if I'm keeping above a relatively low baseline of 150 minutes/week
n_day_avg_workout_duration = Trendsetter.compute_n_sample_avg(all_workouts, DURATION, N_DAYS_TO_AVG)

# Fit relevant trendlines
weight_trendline = Trendsetter.fit_linear(health_metrics, WEIGHT, EXTRAPOLATE_DAYS)
heart_rate_trendline = Trendsetter.fit_logarithmic(health_metrics, RESTING_HEART_RATE, EXTRAPOLATE_DAYS)

### Build Visuals

In [ ]:
# TODO Build at least these visuals:
# * Walking data (max distance, max elevation gain, max duration, pace graph)
# * Strength Metrics (PB + 10-rep; format TBD, likely better as two separate graphs or perhaps grouped by workout)
#    - Ideally: drop-down menu to select between various workouts, each plots its PB and 10-rep over time
ABOVE_TABLE = 1.15

def convert_pd_to_np(obj: NDFrame) -> np.ndarray:
    return np.array(obj)[:, None]

def convert_mins_to_hour_mins(mins: number, _ = None) -> str:
    if mins < 60:
        return f"{int(mins)}m"
    hours, mins = int(mins // 60), int(mins % 60)
    return f"{hours}h {mins}m"

### Utility function, not used in main path ###
def show_gcf_corners(plot: plt):
    plot.gcf().text(0, 0, "x")
    plot.gcf().text(1, 0, "x")
    plot.gcf().text(0, 1, "x")
    plot.gcf().text(1, 1, "x")

def configure_x_axis_by_month(all_workouts: pd.DataFrame, start_padding_days: int=1, end_padding_days: int=1):
    """Sets the current axes x-axis to major tick by month, minor tick on Sundays, and have MMM-YYYY major labels."""
    ax = plt.gca()
    ax.xaxis.set_major_locator(mdates.MonthLocator(interval=1))
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
    ax.xaxis.set_minor_locator(mdates.WeekdayLocator(byweekday=mdates.SU))
    plt.xlim(
        all_workouts[DATE][0] - timedelta(days=start_padding_days),
        all_workouts[DATE].tail(1) + timedelta(days=end_padding_days)
    )

def create_legend_and_title(title: str, reverse_labels: bool=False, ncol: int=2):
    """Adds a legend and title centered above the current plot"""
    plt.title(title, y=ABOVE_TABLE)
    handles, labels = plt.gca().get_legend_handles_labels()
    if reverse_labels:
        handles, labels = handles[::-1], labels[::-1]
    plt.legend(handles, labels, bbox_to_anchor=(0.5, ABOVE_TABLE), loc='upper center', frameon=False, ncol=ncol)

#### Workout Frequency

In [ ]:
RIGHT_OF_AXIS_X_COORD = 0.92
NON_GRAPH_GCF_PERCENT = 0.06

# Draw the main graph contents and setup the axes
workout_durations_mins = all_workouts[DURATION] // 60
plt.scatter(
    all_workouts[DATE],
    workout_durations_mins,
    s=5,
    label="Workout Duration",
)
plt.plot(
    convert_pd_to_np(all_workouts[DATE]),
    convert_pd_to_np(n_day_avg_workout_duration // 60),
    label=f"{N_DAYS_TO_AVG}-Day Avg Daily Duration",
)

# Delineate the ideal minimum daily exercise threshold as a horizontal reference line
plt.axhline(y=MIN_DAILY_ACTIVE_MINUTES, color='r', linestyle='-')
y_percent_min_daily_active = MIN_DAILY_ACTIVE_MINUTES / max(workout_durations_mins)
y_pos = y_percent_min_daily_active + NON_GRAPH_GCF_PERCENT
plt.gcf().text(RIGHT_OF_AXIS_X_COORD, y_pos, "Target\nMinimum")

# Set up axes
ax = plt.gca()
configure_x_axis_by_month(all_workouts)
ax.yaxis.set_major_formatter(ticker.FuncFormatter(convert_mins_to_hour_mins))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(5))
plt.grid(visible=True)
plt.grid(visible=True, which="minor", linestyle="--", linewidth="0.25")

# Add in the surrounding information
ABOVE_TABLE = 1.15
create_legend_and_title("Workout Frequency", reverse_labels=True)
plt.savefig('../img/workout_frequency.png', bbox_inches="tight")
plt.show()

# TODO change colour of rest days to red or orange and MAYBE also color-code cardio vs weights
# Another idea would be generating sub-graphs for each step of the hierarchy. E.g.
# All -> Cardio -> Walk
# All -> Weight -> Chest (per muscle group, not pairs, bc I don't want to marry the visuals to the current splits)

#### Resting Heart Rate

In [ ]:
Y_MIN, Y_MAX = 45, 90
BOTTOM_OFFSET = 0.11
NON_GRAPH_AREA_SCALER = 1.3

def get_padded_dates(df: pd.DataFrame, num_days_to_pad: int):
    """Pads the dates """
    first_index = df.index[0]
    periods = df.shape[0] + num_days_to_pad
    padded_dates = pd.date_range(df.iloc[0][DATE], periods=periods, freq='1d')
    padded_dates = padded_dates.to_series(name=DATE).reset_index(drop=True)
    padded_dates.index = pd.RangeIndex(start=first_index, stop=first_index + periods)
    return padded_dates

nonnull_heart_rates = health_metrics[health_metrics[RESTING_HEART_RATE].notnull()]
padded_dates = get_padded_dates(nonnull_heart_rates, EXTRAPOLATE_DAYS)
plt.scatter(
    nonnull_heart_rates[DATE].to_numpy(),
    nonnull_heart_rates[RESTING_HEART_RATE].to_numpy(),
    s=5,
    label="Resting HR"
)
plt.plot(
    padded_dates.to_numpy(),
    heart_rate_trendline,
    linestyle="--",
    label="Projected Resting HR"
)

# Delineate various resting heart rate levels as horizontal reference lines
resting_heart_rate_levels = {
    "Average": 72,
    "Above Average": 68,
    "Good": 63,
    "Excellent": 58,
    "Athlete": 50,
}
for text, hr in resting_heart_rate_levels.items():
    plt.axhline(y=hr, color='k', linestyle='--', linewidth="0.75")
    y_range = Y_MAX - Y_MIN
    y_pos = BOTTOM_OFFSET + (hr - Y_MIN) / (NON_GRAPH_AREA_SCALER*y_range)
    plt.gcf().text(RIGHT_OF_AXIS_X_COORD, y_pos, text)

# Set up axes
ax = plt.gca()
configure_x_axis_by_month(all_workouts, end_padding_days=EXTRAPOLATE_DAYS)
ax.yaxis.set_minor_locator(ticker.MultipleLocator(1))
ax.set_ylim([Y_MIN, Y_MAX])
plt.grid(visible=True)
plt.grid(visible=True, which="minor", linestyle="--", linewidth="0.25")

# Add in the surrounding information
create_legend_and_title("Resting Heart Rate", reverse_labels=True)
plt.savefig('../img/resting_heart_rate.png', bbox_inches="tight")
plt.show()

#### Weight

In [ ]:
Y_MIN, Y_MAX = 180, 300

nonnull_weights = health_metrics[health_metrics[WEIGHT].notnull()]
padded_dates = get_padded_dates(nonnull_weights, EXTRAPOLATE_DAYS)
plt.scatter(
    nonnull_weights[DATE].to_numpy(),
    nonnull_weights[WEIGHT].to_numpy(),
    s=5,
    label="Weight"
)
plt.plot(
    padded_dates.to_numpy(),
    weight_trendline,
    linestyle="--",
    label="Projected Weight"
)

# Delineate various resting heart rate levels as horizontal reference lines
weight_levels = {
    "Healthy": 250,
    "Target": 200,
}
for text, weight in weight_levels.items():
    plt.axhline(y=weight, color='k', linestyle='--', linewidth="0.75")
    y_range = Y_MAX - Y_MIN
    y_pos = BOTTOM_OFFSET + (weight - Y_MIN) / (NON_GRAPH_AREA_SCALER*y_range)
    plt.gcf().text(RIGHT_OF_AXIS_X_COORD, y_pos, text)

# Set up axes
ax = plt.gca()
configure_x_axis_by_month(all_workouts, end_padding_days=EXTRAPOLATE_DAYS)
ax.yaxis.set_minor_locator(ticker.MultipleLocator(5))
ax.set_ylim([Y_MIN, Y_MAX])
plt.grid(visible=True)
plt.grid(visible=True, which="minor", linestyle="--", linewidth="0.25")

# Add in the surrounding information
create_legend_and_title("Weight", reverse_labels=True)
plt.savefig('../img/weight.png', bbox_inches="tight")
plt.show()

In [ ]:
walk_workouts = cardio_workouts[cardio_workouts[WORKOUT_TYPE].isin({WALK_TREADMILL, WALK_OUTDOOR})]
distance_walked = round(walk_workouts[DISTANCE].to_numpy().sum())
avg_distance = np.average(walk_workouts[DISTANCE].to_numpy())
farthest = walk_workouts[DISTANCE].to_numpy().max()
avg_duration = round(np.average(walk_workouts[DURATION].to_numpy()))
avg_pace = np.average(walk_workouts[PACE].to_numpy())
fastest_pace = walk_workouts[PACE].to_numpy().max()
avg_duration_str = convert_mins_to_hour_mins(avg_duration // 60)
print("Walking Metrics")
print("Average distance: {:.2f}km".format(avg_distance))
print(f"Farthest distance: {farthest}km")
print(f"Total distance: {distance_walked}km")
print(f"Average duration: {avg_duration_str}")
print("Average pace (m/s): {:.2f}".format(avg_pace))
print("Fastest pace (m/s): {:.2f}".format(fastest_pace))

bike_workouts = cardio_workouts[cardio_workouts[WORKOUT_TYPE] == BIKE_STATIONARY]
distance_biked = round(bike_workouts[DISTANCE].to_numpy().sum())
print()
print("Biking Metrics")
print(f"Total distance: {distance_biked}km")

distance_travelled = distance_walked + distance_biked
print()
print("Summary Metrics")
print(f"Total distance travelled: {distance_travelled}km")